In [18]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
#ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
#sys.path.append(ROOT_DIR)  # To find local version of the library
import utils
import visualize
from visualize import display_images
import model as modellib
from model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(os.getcwd(), "trainlogs")

# Local path to trained weights file
#COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
#    utils.download_trained_weights(COCO_MODEL_PATH)

# Path to Shapes trained weights
COCO_MODEL_PATH = os.path.join(os.getcwd(), "cocolayer1output/mask_rcnn_coco_0160.h5")

In [19]:
# Run one of the code blocks

# Shapes toy dataset
# import shapes
# config = shapes.ShapesConfig()

# MS COCO Dataset
import coco
config = coco.CocoConfig()
COCO_DIR = "datasets/cocoL1"  # TODO: enter value here

In [20]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 87

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                100
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE        

In [21]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:1"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [22]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [23]:
# Build validation dataset
if config.NAME == "coco":
    dataset = coco.CocoDataset()
    dataset.load_coco(COCO_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
Images: 4425
Classes: ['BG', '/m/011k07', '/m/0120dh', '/m/01226z', '/m/01599', '/m/01940j', '/m/01bl7v', '/m/01cmb2', '/m/01dws', '/m/01dxs', '/m/01dy8n', '/m/01f8m5', '/m/01j51', '/m/01lcw4', '/m/01s55n', '/m/01yrx', '/m/01yx86', '/m/01z1kdw', '/m/024g6', '/m/025rp__', '/m/027pcv', '/m/029b3', '/m/02fq_6', '/m/02jfl0', '/m/02pv19', '/m/02rgn06', '/m/02vqfm', '/m/02wbtzl', '/m/02zt3', '/m/0306r', '/m/03bt1vf', '/m/03fj2', '/m/03grzl', '/m/03m3pdh', '/m/0449p', '/m/046dlr', '/m/04g2r', '/m/04m9y', '/m/04yqq2', '/m/04yx4', '/m/0584n8', '/m/05_5p_0', '/m/05ctyq', '/m/05n4y', '/m/05r655', '/m/05z6w', '/m/05zsy', '/m/0633h', '/m/06_72j', '/m/06j2d', '/m/06m11', '/m/06pcq', '/m/0703r8', '/m/078jl', '/m/07clx', '/m/07dm6', '/m/080hkjn', '/m/081qc', '/m/096mb', '/m/09b5t', '/m/09csl', '/m/09d5_', '/m/09ddx', '/m/09dzg', '/m/09f_2', '/m/09ld4', '/m/09rvcxw', '/m/0bt9lr', '/m/0by6g', '/m/0c29q', '/m/0ccs93', '/m/

In [24]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# Set weights file path
if config.NAME == "shapes":
    weights_path = SHAPES_MODEL_PATH
elif config.NAME == "coco":
    weights_path = COCO_MODEL_PATH
# Or, uncomment to load the last model you trained
# weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  /home/jupyter/Segmentation/Mask_RCNN/cocolayer1output/mask_rcnn_coco_0160.h5


In [25]:
from pathlib import Path
import pandas as pd
def get_string_to_name():
    csv_path = Path("__file__").parent.joinpath("datasets", "challenge-2019-classes-description-segmentable.csv")
    df = pd.read_csv(str(csv_path), header=None, names=["class_string", "class_name"])
    class_string_to_name = dict(zip(df.class_string,df.class_name))
    return class_string_to_name

In [26]:
string_to_name = get_string_to_name()

In [27]:
classes = ["BG"] 
for i in dataset.class_names:
    if i != "BG":
        classes.append(string_to_name[i])

In [28]:
DATA_DIR = "datasets/coco/train2017"
file_names = next(os.walk(DATA_DIR))[2]

In [ ]:
import random
import skimage.io
# Load a random image from the images folder
#file_names = next(os.walk(DATA_DIR))[2]
#print(file_names)
image = skimage.io.imread(os.path.join(DATA_DIR, random.choice(file_names)))
# image = skimage.io.imread(os.path.join(DATA_DIR, file_names[291]))

# Run detection
results = model.detect([image])

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            classes, r['scores'])

In [14]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib

def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError("encode_binary_mask expects a binary mask, received dtype == %s" % mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError("encode_binary_mask expects a 2d mask, received shape == %s" % mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str


In [15]:
from tqdm import tqdm

In [16]:
empty_submission_df = pd.read_csv("sample_empty_submission.csv")

In [17]:
class_lookup_df = pd.read_csv("challenge-2019-classes-description-segmentable.csv", header=None)

In [16]:
# we have to convert coco classes to this competition's one.

class_lookup_df.columns = ["encoded_label","label"]
# class_lookup_df['label'] = class_lookup_df['label']
class_lookup_df.head()

,encoded_label,label
0,/m/01bms0,Screwdriver
1,/m/03jbxj,Light switch
2,/m/0jy4k,Doughnut
3,/m/09gtd,Toilet paper
4,/m/01j5ks,Wrench


In [19]:
class_lookup_df[class_lookup_df["label"] == "Screwdriver"]

,encoded_label,label
0,/m/01bms0,Screwdriver


In [ ]:
ImageID_list = []
ImageWidth_list = []
ImageHeight_list = []
PredictionString_list = []

for num, row in tqdm(empty_submission_df.iterrows(), total=len(empty_submission_df)):
    filename = row["ImageID"] + ".jpg"
   
    image = skimage.io.imread(os.path.join("datasets/test", filename))
    results = model.detect([image])
    r = results[0]
    
    height = image.shape[0]
    width  = image.shape[1]
        
    PredictionString = ""
    
    for i in range(len(r["class_ids"])):        
        class_id = r["class_ids"][i]
        roi = r["rois"][i]
        mask = r["masks"][:,:,i]
        confidence = r["scores"][i]
        
        encoded_mask = encode_binary_mask(mask)
        
        labelname = classes[r['class_ids'][i]]
        if class_lookup_df[class_lookup_df["label"] == labelname].shape[0] == 0:
            # no match label
            continue
        
        encoded_label = class_lookup_df[class_lookup_df["label"] == labelname]["encoded_label"].item()

        PredictionString += encoded_label 
        PredictionString += " "
        PredictionString += str(confidence)
        PredictionString += " "
        PredictionString += encoded_mask.decode()
        PredictionString += " "
        
    ImageID_list.append(row["ImageID"])
    ImageWidth_list.append(width)
    ImageHeight_list.append(height)
    PredictionString_list.append(PredictionString)

 67%|██████▋   | 66966/99999 [7:22:06<3:52:04,  2.37it/s]

In [ ]:
results=pd.DataFrame({"ImageID":ImageID_list,
                      "ImageWidth":ImageWidth_list,
                      "ImageHeight":ImageHeight_list,
                      "PredictionString":PredictionString_list
                     })

In [ ]:
results.to_csv("submissionL1.csv", index=False)

In [ ]:
import pandas as pd
df = pd.read_csv("submissionL1.csv")
df.head()